In [334]:
# ovo pokrenuti ukoliko želite da se svi izrazi unutar ćelije ispišu
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# N-gramski jezični model

In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize
from nltk.lm.preprocessing import pad_both_ends, flatten, padded_everygram_pipeline
from nltk.util import bigrams, ngrams
from nltk.lm import MLE, Laplace, Vocabulary
import nltk.lm as lm
import random
import re

In [2]:
text =  [['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f']]

In [3]:
from nltk.util import bigrams, ngrams
list(bigrams(text[0]))
[list(ngrams(sent,n=3)) for sent in text]

[[('a', 'b', 'c')],
 [('a', 'c', 'd'), ('c', 'd', 'c'), ('d', 'c', 'e'), ('c', 'e', 'f')]]

In [4]:
from nltk.util import pad_sequence
list(pad_sequence(text[0],
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
n=2))

['<s>', 'a', 'b', 'c', '</s>']

In [5]:
list(bigrams(pad_both_ends(text[0], n=2)))

[('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>')]

Za model nam  je potreban skup svih tokena kao vokabular.

In [6]:
from nltk.lm.preprocessing import flatten
vocab = list(flatten(pad_both_ends(sent, n=2) for sent in text))
vocab

['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']

In [7]:
from nltk.lm.preprocessing import padded_everygram_pipeline


train, vocab = padded_everygram_pipeline(2, text)
#train = [list(ngrams(pad_both_ends(sent, n=2),2)) for sent in text]
#vocab = list(flatten(pad_both_ends(sent, n=2) for sent in text))

print('text',text)
# koristimo pipeline u kojem su train i vocab generatori
print('train',list(list(train)[0]))
print('vocab',list(vocab))





text [['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f']]
train [('<s>',), ('<s>', 'a'), ('a',), ('a', 'b'), ('b',), ('b', 'c'), ('c',), ('c', '</s>'), ('</s>',)]
vocab ['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


In [8]:
text = [['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f'],['a','f','e']]
test = [('a', 'b'), ('c', 'd')]

from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, text)


from nltk.lm import MLE
lm = MLE(2)
len(lm.vocab)

lm.fit(train, vocab)
len(lm.vocab)

lm.vocab.lookup(text[0])
print(lm.counts)

# koliko puta se pojavilo 'a'
lm.counts['a']

# koliko se pouta pojavilo 'a b' kao bigram
lm.counts[['a']]['b']

# vjerojatnost pridružena modelom 
lm.score('a')

# koliko puta se pojavio 'b' nakon 'a' kao bigram
lm.score("b", ["a"])

lm.entropy(test)
lm.perplexity(test)

<NgramCounter with 2 ngram orders and 33 ngrams>


3.0000000000000004

## Primjer s korpusom teksta

In [318]:
def prepare(corpora,split=0.7):    
    dataset = open(corpora,'r').read()
    #dataset = re.sub('\n',dataset,'.')
    sents = nltk.tokenize.sent_tokenize(dataset)
    #pdb.set_trace()
    random.shuffle(sents)
    index = random.randint(0, int(split*len(sents)))
    trainset, testset = sents[index:], sents[:index]
    
    # store training set
    with open('data/train.txt','w') as train:
        for sent in trainset:
            train.write(sent)
            train.write('\n')
        
    # store test set
    with open('data/test.txt','w') as test:
        for sent in testset: 
            test.write(sent)
            test.write('\n')

In [319]:
prepare('data/lektira/werther.txt')

In [346]:
PlaintextCorpusReader('data/','train.txt').sents()


[['Rado', 'ti', 'priznajem', '(', 'jer', 'znam', 'što', 'bi', 'mi', 'rekao', 'na', 'to', ')', 'da', 'su', 'najsretniji', 'oni', 'koji', ',', 'kao', 'djeca', ',', 'žive', 'od', 'dana', 'u', 'dan', ',', 'vuku', 'sa', 'sobom', 'svoje', 'lutke', ',', 'svlače', 'ih', 'i', 'oblače', 'i', 's', 'velikim', 'se', 'respektom', 'kradu', 'oko', 'ladice', 'u', 'koju', 'je', 'mamica', 'zaključala', 'slatkiš', ',', 'a', 'kad', 'napokon', 'ujagme', 'ono', 'što', 'žele', ',', 'gutaju', 'punim', 'ustima', 'i', 'viču', ':', 'Još', '!'], ['*'], ...]

In [347]:
PlaintextCorpusReader('data/','test.txt').sents()


[['Povučem', 'ih', 'nazad', 'kao', 'od', 'vatre', ',', 'a', 'neka', 'tajna', 'sila', 'vuče', 'me', 'opet', 'naprijed', '—', 'nesvjestica', 'me', 'spopada', 'u', 'svim', 'ćutilima', '.'], ['A', 'zašto', 'ne', 'bi', '?'], ...]

In [352]:
train_set = PlaintextCorpusReader('data/','train.txt').sents()
test_set  = PlaintextCorpusReader('data/','test.txt').sents()

# ngrams
nn = 2
       
# normaliziraj tokene
train_set = [[token.lower() for token in sent] for sent in train_set ]
test_set  =  [[token.lower() for token in sent] for sent in test_set ]

test = [list(ngrams(sent,nn)) for sent in test_set][0]


from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(nn, train_set)




In [349]:
train_set[0]
test_set[0]

['rado',
 'ti',
 'priznajem',
 '(',
 'jer',
 'znam',
 'što',
 'bi',
 'mi',
 'rekao',
 'na',
 'to',
 ')',
 'da',
 'su',
 'najsretniji',
 'oni',
 'koji',
 ',',
 'kao',
 'djeca',
 ',',
 'žive',
 'od',
 'dana',
 'u',
 'dan',
 ',',
 'vuku',
 'sa',
 'sobom',
 'svoje',
 'lutke',
 ',',
 'svlače',
 'ih',
 'i',
 'oblače',
 'i',
 's',
 'velikim',
 'se',
 'respektom',
 'kradu',
 'oko',
 'ladice',
 'u',
 'koju',
 'je',
 'mamica',
 'zaključala',
 'slatkiš',
 ',',
 'a',
 'kad',
 'napokon',
 'ujagme',
 'ono',
 'što',
 'žele',
 ',',
 'gutaju',
 'punim',
 'ustima',
 'i',
 'viču',
 ':',
 'još',
 '!']

['povučem',
 'ih',
 'nazad',
 'kao',
 'od',
 'vatre',
 ',',
 'a',
 'neka',
 'tajna',
 'sila',
 'vuče',
 'me',
 'opet',
 'naprijed',
 '—',
 'nesvjestica',
 'me',
 'spopada',
 'u',
 'svim',
 'ćutilima',
 '.']

Treniranje primjera iz vlastitog korpusa.

In [353]:

from nltk.lm import MLE, Laplace
lm = MLE(nn)
#lm = Laplace(nn)
lm.vocab

# velicina n-grama u modelu
print(lm.order)
# treniraj model
lm.fit(train, vocab)
# nova velicina vokabulara
len(lm.vocab)
# velicine
print(lm.counts)

lm.vocab.lookup('bio'), 
lm.counts[['bio']]['je']


lm.entropy(test)
lm.perplexity(test)
lm.generate(10)

2


10091

<NgramCounter with 2 ngram orders and 100968 ngrams>


('bio',)

18

12.833666754980955

7299.929192712028

['sila',
 ':',
 'prisegao',
 'sam',
 'portret',
 'započeo',
 'triput',
 'sam',
 'već',
 'se']